In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch 

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms

from PIL import Image

from math import floor

import torch.nn.functional as F

from sklearn.preprocessing import MultiLabelBinarizer

import os

import torchvision.models as models

%matplotlib inline

In [2]:
csv_data = pd.read_csv("train_v2.csv")
#data = np.array(np.zeros((500,256,256,3)),dtype=np.float32)

In [3]:
#def DataLoader(data,batch_size = 10):
#    i = 0
#    ret_data = []
#    while True:
#        
#        if i+batch_size > images_names.shape[0]:
#            get_data = images_names[i:]
#            i=0
#        else:
#            get_data = images_names[i:i+batch_size]
#            i += batch_size
#            
#        for image in get_data:
#            ret_data.append(plt.imread("train-jpg/%s.jpg" % image)[:,:,0:3].reshape(3,256,256))#
#
#        
#        yield torch.from_numpy(np.array(ret_data).astype('float32')).type(torch.FloatTensor)

In [4]:
def train_valid_split(dataframe, test_size = 0.25, shuffle = False, random_seed = 0):
    """ Return a list of splitted indices from a DataSet.
    Indices can be used with DataLoader to build a train and validation set.
    
    Arguments:
        A Dataframe
        A test_size, as a float between 0 and 1 (percentage split) or as an int (fixed number split)
        Shuffling True or False
        Random seed
    """
    length = len(dataframe)
    indices = list(range(1,length))
    
    if shuffle == True:
        random.seed(random_seed)
        random.shuffle(indices)
    
    if type(test_size) is float:
        split = floor(test_size * length)
    elif type(test_size) is int:
        split = test_size
    else:
        raise ValueError('%s should be an int or a float' % str)
    return indices[split:], indices[:split]

In [5]:
class KaggleAmazonDataset(Dataset):
    """Dataset wrapping images and target labels for Kaggle - Planet Amazon from Space competition.

    Arguments:
        A CSV file path
        Path to image folder
        Extension of images
        PIL transforms
    """

    def __init__(self, csv_path, img_path, img_ext, transform=None):
    
        tmp_df = pd.read_csv(csv_path)
        assert tmp_df['image_name'].apply(lambda x: os.path.isfile(img_path + x + img_ext)).all(), \
"Some images referenced in the CSV file were not found"
        
        self.mlb = MultiLabelBinarizer()
        self.img_path = img_path
        self.img_ext = img_ext
        self.transform = transform

        self.X_train = tmp_df['image_name']
        self.y_train = self.mlb.fit_transform(tmp_df['tags'].str.split()).astype(np.float32)

    def __getitem__(self, index):
        img = Image.open(self.img_path + self.X_train[index] + self.img_ext)
        img = img.convert('RGB')
        if self.transform is not None:
            img = self.transform(img).cuda()
        
        label = torch.from_numpy(self.y_train[index]).cuda()
        return img, label

    def __len__(self):
        return len(self.X_train.index)

In [6]:
#data = DataLoader(csv_data["image_name"])
#label = DataLoader(csv_data["tags"])
#for i,data in enumerate(data):
    #print(i),print(data)
IMG_PATH = 'train-jpg/'
IMG_EXT = '.jpg'
TRAIN_DATA = 'train_v2.csv'
#transformations = transforms.Compose([transforms.Scale(32),transforms.ToTensor()])
#dset_train = KaggleAmazonDataset(TRAIN_DATA,IMG_PATH,IMG_EXT,transformations)


## Augmentation + Normalization for full training
ds_transform_augmented = transforms.Compose([
                 transforms.RandomSizedCrop(224),
                 #transforms.Scale(512),
                 transforms.RandomHorizontalFlip(), 
                 transforms.ToTensor(),
                 #transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                # std=[0.229, 0.224, 0.225]),

                 # Affine(
                 #     rotation_range = 15,
                 #     translation_range = (0.2,0.2),
                 #     shear_range = math.pi/6,
                 #     zoom_range=(0.7,1.4)
                 # )
                ])

## Normalization only for validation and test
ds_transform_raw = transforms.Compose([
                 transforms.Scale(224),
                 transforms.ToTensor(),
                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.02, 0.02, 0.225])
                 ])

####     #########     ########     ###########     #####

X_train = KaggleAmazonDataset(TRAIN_DATA,IMG_PATH,IMG_EXT,
                             ds_transform_augmented
                             )
X_val = KaggleAmazonDataset(TRAIN_DATA,IMG_PATH,IMG_EXT,
                             ds_transform_raw
                             )

In [7]:
# Creating a validation split
train_idx, valid_idx = train_valid_split(X_train, 0.2)

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = DataLoader(X_train,
                          batch_size=2,
                          sampler=train_sampler,
                          #num_workers=4,
                          #pin_memory= True
                         )

valid_loader = DataLoader(X_val,
                          batch_size=2,
                          sampler=valid_sampler,
                          #num_workers=4,
                         #pin_memory= True
                         )

In [8]:
def adaptive_avgmax_pool2d(x, pool_type='avg', padding=0, count_include_pad=False):
    """Selectable global pooling function with dynamic input kernel size
    """
    if pool_type == 'avgmaxc':
        x = torch.cat([
            F.avg_pool2d(
                x, kernel_size=(x.size(2), x.size(3)), padding=padding, count_include_pad=count_include_pad),
            F.max_pool2d(x, kernel_size=(x.size(2), x.size(3)), padding=padding)
        ], dim=1)
    elif pool_type == 'avgmax':
        x_avg = F.avg_pool2d(
                x, kernel_size=(x.size(2), x.size(3)), padding=padding, count_include_pad=count_include_pad)
        x_max = F.max_pool2d(x, kernel_size=(x.size(2), x.size(3)), padding=padding)
        x = 0.5 * (x_avg + x_max)
    elif pool_type == 'max':
        x = F.max_pool2d(x, kernel_size=(x.size(2), x.size(3)), padding=padding)
    else:
        if pool_type != 'avg':
            print('Invalid pool type %s specified. Defaulting to average pooling.' % pool_type)
        x = F.avg_pool2d(
            x, kernel_size=(x.size(2), x.size(3)), padding=padding, count_include_pad=count_include_pad)
    return x

def pooling_factor(pool_type='avg'):
    return 2 if pool_type == 'avgmaxc' else 1


import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import torch.utils.model_zoo as model_zoo

__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152']

model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}


def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, drop_rate=0.0):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride
        self.drop_rate = drop_rate

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        if self.drop_rate > 0.:
            out = F.dropout(out, p=self.drop_rate, training=self.training)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, drop_rate=0.0):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        self.drop_rate = drop_rate

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        if self.drop_rate > 0.:
            out = F.dropout(out, p=self.drop_rate, training=self.training)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000,
                 drop_rate=0.0, block_drop_rate=0.0,
                 global_pool='avg'):
        self.inplanes = 64
        self.drop_rate = drop_rate
        self.expansion = block.expansion
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], drop_rate=block_drop_rate)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, drop_rate=block_drop_rate)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, drop_rate=block_drop_rate)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, drop_rate=block_drop_rate)
        self.global_pool = AdaptiveAvgMaxPool2d(pool_type=global_pool)
        self.fc = nn.Linear(512 * block.expansion * self.global_pool.factor(), num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1, drop_rate=0.):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = [block(self.inplanes, planes, stride, downsample, drop_rate)]
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def get_fc(self):
        return self.fc

    def reset_fc(self, num_classes, global_pool='avg'):
        self.global_pool = AdaptiveAvgMaxPool2d(pool_type=global_pool)
        self.fc = nn.Linear(512 * self.expansion * self.global_pool.factor(), num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.global_pool(x)
        x = x.view(x.size(0), -1)

        if self.drop_rate > 0.:
            x = F.dropout(x, p=self.drop_rate, training=self.training)

        x = self.fc(x)
        return x


def resnet18(pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model


def resnet34(pretrained=False, **kwargs):
    """Constructs a ResNet-34 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet34']))
    return model


def resnet50(pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
    return model


def resnet101(pretrained=False, **kwargs):
    """Constructs a ResNet-101 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet101']))
    return model


def resnet152(pretrained=False, **kwargs):
    """Constructs a ResNet-152 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet152']))
    return model


In [10]:
class AdaptiveAvgMaxPool2d(torch.nn.Module):
    """Selectable global pooling layer with dynamic input kernel size
    """
    def __init__(self, output_size=1, pool_type='avg'):
        super(AdaptiveAvgMaxPool2d, self).__init__()
        self.output_size = output_size
        self.pool_type = pool_type
        if pool_type == 'avgmaxc' or pool_type == 'avgmax':
            self.pool = nn.ModuleList([nn.AdaptiveAvgPool2d(output_size), nn.AdaptiveMaxPool2d(output_size)])
        elif pool_type == 'max':
            self.pool = nn.AdaptiveMaxPool2d(output_size)
        else:
            if pool_type != 'avg':
                print('Invalid pool type %s specified. Defaulting to average pooling.' % pool_type)
            self.pool = nn.AdaptiveAvgPool2d(output_size)

    def forward(self, x):
        if self.pool_type == 'avgmaxc':
            x = torch.cat([p(x) for p in self.pool], dim=1)
        elif self.pool_type == 'avgmax':
            x = 0.5 * torch.sum(torch.stack([p(x) for p in self.pool]), 0).squeeze(dim=0)
        else:
            x = self.pool(x)
        return x

    def factor(self):
        return pooling_factor(self.pool_type)

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + 'output_size=' + str(self.output_size) \
+ ', pool_type=' + self.pool_type + ')'

#model = Net()
#model = models.inception_v3(num_classes=17, aux_logits=False, transform_input = True)
#model = models.vgg16(pretrained=True, num_classes=17)
model = resnet50(num_classes=17)
optimizer = optim.RMSprop(model.parameters(),lr=0.00001)
optimizer = optim.Adam(model.parameters(),lr=0.0001)
criterion = nn.MultiLabelSoftMarginLoss()

In [14]:
from sklearn.metrics import fbeta_score

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    return fbeta_score(output, target, beta=2, average="samples")

def train_generator(model,epoch,train_loader,optimizer,criterion):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            #data, target = data.cuda(async=True), target.cuda(async=True) # On GPU
            data, target = Variable(data), Variable(target)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % 50 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.data[0]))
                
def test_generator(model, val_loader, criterion):
        model.eval()
        test_loss = 0
        ammount = 0
        equal = 0
        pred_all = None
        test_all = None
        test = 0
        for data, target in val_loader:
            data, target = Variable(data.float(), volatile=True), Variable(target.float())
            output = model(data)
            ammount += len(data)
            test_loss += criterion(output, target).data[0]
            pred = output
            pred[pred > 0.5] = 1
            pred[pred != 1] = 0

            if type(pred_all) == type(None):
                pred_all = pred.cpu().data.numpy().astype(np.uint8)
            else:
                pred = pred.cpu().data.numpy().astype(np.uint8)

                pred_all = np.concatenate((pred_all,pred),axis=0)
            if type(test_all) == type(None):
                test_all = target.cpu().data.numpy().astype(np.uint8)
            else:
                target = target.cpu().data.numpy().astype(np.uint8)
                test_all = np.concatenate((test_all,target),axis=0)
            test += 1
            #accuracy(output, target,topk=(1,3))
            if test > 10:
                break
        test_loss /= ammount
        print('Test set: Accuracy: %s\n' % accuracy(pred_all,test_all))

In [ ]:
model = torch.nn.DataParallel(model).cuda()
for epoch in range(1, 10 + 1):
    train_generator(model,epoch,train_loader,optimizer,criterion)
    test_generator(model,valid_loader,criterion)

Train Epoch: 1 [0/40479 (0%)]	Loss: 0.302637
Train Epoch: 1 [100/40479 (0%)]	Loss: 0.249423
Train Epoch: 1 [200/40479 (1%)]	Loss: 0.190985
Train Epoch: 1 [300/40479 (1%)]	Loss: 0.403225
